# Assemble the final Dataframe from tarpmir bindingsites

## Read in bindingsite & Ensembl data

In [1]:
%load_ext autoreload
%autoreload 2
from helper_fcts import *
path_tarp = 'C:/Users/Lena/Documents/Master big files/TarPmiR_Windows/data/'
path_gdc = 'C:/Users/Lena/Documents/Master big files/GDC_download_legacy/'
path2 = 'C:/Users/Lena/Documents/auf mega gesyncte Dokumente/Uni/Master/Semester 5/Masterarbeit/'
fig_path = 'Dokumente/plots/'

In [3]:
all_mapping, ids = read_in_ids2(path_tarp+'all_mapping_ids.fasta') # mapping gene, transcript, exon ids, all
chrom_exon_starts = read_in_fasta2(path_tarp+'all_exon_start.fasta')
chrom_exon_ends = read_in_fasta2(path_tarp+'all_exon_end.fasta')
exon_starts, exon_ends = calc_exon_data(chrom_exon_starts, chrom_exon_ends)#TODO remove this & instead work with exon_info? slower :(
binding_sites, bs_starts, bs_ends = parse_tarp_bs(path_tarp+'new_input/input_miRNA.fasta_exampleA1BG_mRNA.txt.bp')

#df_main: bs_id - chr_start - chr_end - miRNA_name - bs_seed - bs_len - tarp_output - transcript_id
#df_gene: strand - chrom - gene_id - gene_name - transcript_id (1-N)

In [4]:
#parse miRNA and mRNA seed from sequence
df_main = binding_sites.copy()
miRNA_sequences = parse_seq(path_tarp+'new_input/input_miRNA.fasta')
df_main['miRNA_seed'] = df_main.apply(lambda row: miRNA_sequences[row['miRNA']], axis=1)
mRNA_sequences = parse_seq(path_tarp+'new_input/exampleA1BG_mRNA.txt')
df_main['mRNA_bs_seq'] = df_main.apply(lambda row: mRNA_sequences[row['mRNA']][row['bs_start']:row['bs_end']], axis=1)

In [5]:
#translate genome position relative to transcript to chromosome
def pos_to_chrom(pos, tid, ids, exon_starts, exon_ends, chrom_exon_starts):
    for j, eid in enumerate(ids[tid]):
        if tid not in exon_starts: print(tid,'not in exon_starts')
        if tid in exon_starts and eid in exon_starts[tid] and tid in exon_ends and eid in exon_ends[tid]:
            if pos >= exon_starts[tid][eid] and pos <= exon_ends[tid][eid]:
                diff = pos - exon_starts[tid][eid]
                chrom_pos = chrom_exon_starts[tid][eid] + diff
                return chrom_pos
        else:
            print(eid,'not in exon_starts')
            return None
df_main['chrom_bs_start'] = df_main.apply(lambda row: pos_to_chrom(row.bs_start, row.mRNA, ids, exon_starts, exon_ends, chrom_exon_starts), axis=1)
df_main['chrom_bs_end'] = df_main.apply(lambda row: pos_to_chrom(row.bs_end, row.mRNA, ids, exon_starts, exon_ends, chrom_exon_starts), axis=1)
df_main['bs_id'] = df_main.index
df_main.head()

,miRNA,mRNA,binding_probability,energy,seed,accessibility,AU_content,PhyloP_Stem,PyloP_Flanking,m/e,...,position_of_longest_consecutive_pairings,pairings_in_3prime_end,difference_of_pairings_between_seed_and_3prime_end,bs_start,bs_end,miRNA_seed,mRNA_bs_seq,chrom_bs_start,chrom_bs_end,bs_id
0,hsa-let-7a-5p,ENST00000600966,0.692308,-15.4,0,0.003371,0.368,-0.568203,0.198163,-7.757047,...,10,6,1,396,416,TGAGGTAGTAGGTTGTATAGTT,AACTACAGCTGCAGCTACCG,58352138,58352158,0
1,hsa-let-7a-2-3p,ENST00000600966,0.923077,-23.2,1,0.000094,0.426,0.844482,-0.095065,-5.585164,...,1,6,1,52,75,CTGTACAGCCTCCTAGCTTTCC,GGGTGGCCCAGGAGCCTGTGCAC,58350645,58351407,1
2,hsa-let-7b-5p,ENST00000600966,0.692308,-21.3,0,0.000009,0.353,1.253080,0.015255,-8.613887,...,1,7,2,775,794,TGAGGTAGTAGGTTGTGTGGTT,ACTACACCTGCCGCTACCG,58352987,58353006,2
3,hsa-let-7b-5p,ENST00000600966,0.692308,-18.5,0,0.000001,0.294,1.302461,-0.092658,-9.582385,...,1,6,1,126,143,TGAGGTAGTAGGTTGTGTGGTT,GGCCGCTACCGCTGCCG,58351458,58351475,3
4,hsa-let-7b-5p,ENST00000600966,0.846154,-19.2,0,0.000174,0.397,0.062334,0.643183,-8.600034,...,12,8,3,560,580,TGAGGTAGTAGGTTGTGTGGTT,GGCTGCACCACCACCGCCTG,58352400,58352420,4


In [11]:
#df_gene: strand - chrom - gene_id - gene_name - transcript_id (1-N)
df_gene = pd.read_csv(path2+'gene_infos.csv', dtype={'ensembl_gene_id':str,'ensembl_gene_id_version':str,'ensembl_transcript_id':str, 'ensembl_transcript_id_version':str, 'chromosome_name':'category', 'strand':'int8', 'start_position':int, 'end_position':int})
del df_gene['Unnamed: 0']
df_gene.head()

,ensembl_gene_id,ensembl_gene_id_version,ensembl_transcript_id,ensembl_transcript_id_version,chromosome_name,strand,start_position,end_position
0,ENSG00000210049,ENSG00000210049.1,ENST00000387314,ENST00000387314.1,MT,1,577,647
1,ENSG00000211459,ENSG00000211459.2,ENST00000389680,ENST00000389680.2,MT,1,648,1601
2,ENSG00000210077,ENSG00000210077.1,ENST00000387342,ENST00000387342.1,MT,1,1602,1670
3,ENSG00000210082,ENSG00000210082.2,ENST00000387347,ENST00000387347.2,MT,1,1671,3229
4,ENSG00000209082,ENSG00000209082.1,ENST00000386347,ENST00000386347.1,MT,1,3230,3304


## get TCGA data from GDC API 

In [9]:
%load_ext Cython

In [ ]:
%%cython
def c_split_chrom_column(chrom):
    chrom_list = chrom.split(':')
    chromosome_name = str(chrom_list[0][3:])
    strand = 1 if chrom_list[2] == '+' else -1
    chrom_exon_start = int(chrom_list[1].split('-')[0])
    chrom_exon_end = int(chrom_list[1].split('-')[1])
    return chromosome_name,strand,chrom_exon_start,chrom_exon_end

In [ ]:
def list_to_df(list_, nr_columns, data_type=None):
    data = []
    remove_last_columns = 2 if data_type != None else 0
    nr_rows = int((len(list_)-1)/nr_columns)
    for row in range(nr_rows):
        start = row*nr_columns
        end = (row+1)*nr_columns
        data.append(list_[start:end-remove_last_columns])
    df = pd.DataFrame(data[1:], columns = data[0]) #use first line as header
    if data_type=='miRNA':
        df = df.astype({'miRNA_ID':str, 'read_count':int})#, 'reads_per_million_miRNA_mapped':float, 'cross-mapped': 'category'})
    elif data_type=='exon':
        df = df.astype({'exon':str, 'raw_counts':int})#, 'reads_per_million_miRNA_mapped':float, 'cross-mapped': 'category'})
        df[['chromosome_name','strand','chrom_exon_start','chrom_exon_end']] = df.exon.apply(lambda x: pd.Series(c_split_chrom_column(str(x))))
        df[['raw_counts','chrom_exon_start','chrom_exon_end']] = df[['raw_counts','chrom_exon_start','chrom_exon_end']].apply(pd.to_numeric,downcast='unsigned')
        df[['chromosome_name','strand']] = df[['chromosome_name','strand']].astype('str').astype('category')
        del df['exon']
    return df

In [ ]:
#legacy API, get list of files which contain miRNA expression per case_id there
import requests
import json
import re
import gzip

def download_files_from_legacy(data_type, size, primary_site,write = False):
    files_data_type = 'miRNA gene quantification' if data_type == 'miRNA' else 'Exon quantification'
    fields = [
        "file_name",
        "cases.disease_type",
        "cases.case_id",
        "data_category",
        "data_type",
        "cases.project.primary_site"
        ]

    fields = ",".join(fields)

    files_endpt = "https://api.gdc.cancer.gov/legacy/files"

    # This set of filters is nested under an 'and' operator.
    filters = {
        "op": "and",
        "content":[
            {
            "op": "in",
            "content":{
                "field": "cases.project.primary_site",
                "value": [primary_site]
                }
            },
            {
            "op": "in",
            "content":{
                "field": "files.data_type",
                "value": [files_data_type]
                }
            }
        ]
    }

    # A POST is used, so the filter parameters can be passed directly as a Dict object.
    params = {
        "filters": filters,
        "fields": fields,
        "format": "TSV",
        "size": str(size)
        }

    # The parameters are passed to 'json' rather than 'params' in this case
    response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)
    df_files = list_to_df(response.content.decode("utf-8").replace("\r\n", '\t').split('\t'),7)
    all_dfs = {}

    for i,row in df_files.iterrows():
        file_id = row.id
        case_id = row['cases.0.case_id']
        data_endpt = f"https://api.gdc.cancer.gov/legacy/data/{file_id}"

        response = requests.get(data_endpt, headers = {"Content-Type": "application/json"})
        response_head_cd = response.headers["Content-Disposition"]
        data = str(response.content,'utf-8')
        data = data.replace('\n', '\t').split('\t')
        new_df = list_to_df(data,4,data_type)
        if write:
            file_name = f'{path_gdc}{data_type}/{(re.findall("filename=(.+)", response_head_cd)[0])}'
            if not file_name.exists():
                new_df.to_feather(file_name)
        #id_dict = ('.').join(file_name.split('.')[0:3])
        #all_dfs[id_dict] = new_df
        print(i+1,'/',len(df_files),'done.')
    #return all_dfs
primary_site='Kidney'
download_files_from_legacy('miRNA',3000,primary_site,write=True) #2029 #70min
#download_files_from_legacy('exon',2000,primary_site,write=True) #1588 #35h
#takes long & needs internet (DONE: all mirna_dfs (but weird only 873 items but 2019 files, maybe case_id not id), 248 / 1588 exon_dfs)
#normal 1min 44s - #cython 1min 42s

In [6]:
#read in downloaded counts from feather files
#TODO remove later when all data there
import os
n_all_mirna_dfs = {}
dir_ = os.path.join(path_gdc,'mirna')
for i, file in enumerate(os.listdir(dir_)):
    #print(file)
    filename = ('.').join(file.split('.')[0:3])
    n_all_mirna_dfs[filename] = pd.read_feather(os.path.join(dir_,file))
    
n_all_exon_dfs = {}
dir_ = os.path.join(path_gdc,'exon')
for i, file in enumerate(os.listdir(dir_)):
    #print(file)
    filename = ('.').join(file.split('.')[0:3])
    n_all_exon_dfs[filename] = pd.read_feather(os.path.join(dir_,file))
print(len(n_all_exon_dfs),len(n_all_mirna_dfs)) #yes this is right, read in from path
all_mirna_dfs = n_all_mirna_dfs
all_exon_dfs = n_all_exon_dfs

1588 2013


In [242]:
#print, proof that caseid not unique
#TODO remove later when all data there
#find id for files - not only caseid: caseid + 
def print_files_from_legacy(data_type, size, primary_site,write = False):
    files_data_type = 'miRNA gene quantification' if data_type == 'miRNA' else 'Exon quantification'
    fields = [
        "file_name",
        "cases.disease_type",
        "cases.case_id",
        "data_category",
        "data_type",
        "cases.project.primary_site"
        ]

    fields = ",".join(fields)

    files_endpt = "https://api.gdc.cancer.gov/legacy/files"

    # This set of filters is nested under an 'and' operator.
    filters = {
        "op": "and",
        "content":[
            {
            "op": "in",
            "content":{
                "field": "cases.project.primary_site",
                "value": [primary_site]
                }
            },
            {
            "op": "in",
            "content":{
                "field": "files.data_type",
                "value": [files_data_type]
                }
            }
        ]
    }

    # A POST is used, so the filter parameters can be passed directly as a Dict object.
    params = {
        "filters": filters,
        "fields": fields,
        "format": "TSV",
        "size": str(size)
        }

    # The parameters are passed to 'json' rather than 'params' in this case
    response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)
    df_files = list_to_df(response.content.decode("utf-8").replace("\r\n", '\t').split('\t'),7)
    all_dfs = {}
    jo = []
    for i,row in df_files.iterrows():
        file_id = row.id
        case_id = row['cases.0.case_id']
        #print(i+1,file_id,case_id)
        jo.append(case_id)
    print(len(jo),len(set(jo)))
    
primary_site='Kidney'
all_mirna_dfs = print_files_from_legacy('miRNA',3000,'Kidney') #2029 #70min
all_exon_dfs = print_files_from_legacy('exon',2000,'Kidney') #1588 #53h


2029 873
1588 889


In [ ]:
#save dictionaries to file
import pickle
path = 'intermed_save'
#f = open(path+"all_mirna_dfs.pkl","wb")
#pickle.dump(all_mirna_dfs,f)
#f.close()
path = 'intermed_save'
#f = open(path+"all_exon_dfs.pkl","wb")
#pickle.dump(all_exon_dfs,f)
#f.close()

In [239]:
#load dictionaries from file
import pickle
with open(path_gdc+'miRNA/all_mirna_dfs.pkl', 'rb') as handle:
    all_mirna_dfs = pickle.load(handle)
with open(path_gdc+'exon/all_exon_dfs.pkl', 'rb') as handle:
    all_exon_dfs = pickle.load(handle)
print(len(all_exon_dfs),len(all_mirna_dfs))

87 87


In [12]:
#TODO for every case do
#for _,row in df_exons.iterrows():
#case_id =  row['cases.0.case_id']

case_id = 'unc.edu.84675636-1d8c-474a-a4bf-baffcc36b5bf'
important_counts = df_main[['bs_id','miRNA','mRNA','chrom_bs_start','chrom_bs_end']].copy()
important_counts = important_counts.merge(df_gene[['ensembl_transcript_id','chromosome_name','strand']], left_on='mRNA', right_on='ensembl_transcript_id', how='left')
important_counts[['chromosome_name','strand','miRNA','mRNA']] = important_counts[['chromosome_name','strand','miRNA','mRNA']].astype('str').astype('category')
del important_counts['ensembl_transcript_id']

## translate TCGA exon counts to GRCh38

In [18]:
exon_counts = all_exon_dfs[case_id]

In [ ]:
#write exoncounts to BED file for translating GRCh37 to GRCh38

file = open('chrom_pos_hg19_'+primary_site+'.bed', 'w')
for row in exon_counts.iterrows():
    line = row[0]
    sep = '\t'
    strand = '+' if (row[1].strand=='1') else '-'
    start = row[1].chrom_exon_start if strand == '+' else row[1].chrom_exon_end
    end = row[1].chrom_exon_end if strand == '+' else row[1].chrom_exon_start
    string = f'chr{row[1].chromosome_name}{sep}{start}{sep}{end}{sep}{line}{sep}0{sep}{strand}\n' 
    #chr7  127471196  127472363  Pos1  0  +
    file.write(string)
file.close()
#TODO then upload to website

In [ ]:
import os #pyliftover doesnt work cause doesnt work with ranges
import subprocess
if os.name == 'posix':
    bashCommand = "liftOver chrom_pos_hg19_"+primary_site+".bed hg19ToHg37.over.chain.gz chrom_pos_translated.bed unlifted.bed"
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    print(output, error)
elif os.name == 'nt':
    print("liftOver doesn't exist for Windows, manually upload BED file to website")

In [13]:
#then read in new liftover exoncounts in GRCh38 from files
df_liftover = pd.read_csv('chrom_pos_translated.bed', sep="\t",names=['chromosome_name','chrom_exon_start','chrom_exon_end','index','0','strand'])
df_liftover = df_liftover.set_index('index')
df_liftover.chromosome_name = df_liftover.apply(lambda row: row.chromosome_name[3:], axis=1)
df_liftover.strand = df_liftover.apply(lambda row: '1' if (row.strand=='+') else '-1', axis=1)
del df_liftover['0']
df_liftover.head()

,chromosome_name,chrom_exon_start,chrom_exon_end,strand
index,,,,
0,1,11874,12227,1
1,1,12595,12721,1
2,1,12613,12721,1
3,1,12646,12697,1
4,1,13221,14409,1


In [19]:
# changes of exon_counts (GRCh37) to new_exon_counts(GRCh38)
new_exon_counts = df_liftover.join(exon_counts,rsuffix='r',lsuffix='l')#['raw_counts'])
#print('indices, which got lost during translation with liftover:',set(range(239885))-set(new_exon_counts.index))
new_exon_counts = df_liftover.join(exon_counts['raw_counts'])
new_exon_counts.head()

,chromosome_name,chrom_exon_start,chrom_exon_end,strand,raw_counts
index,,,,,
0,1,11874,12227,1,15.0
1,1,12595,12721,1,3.0
2,1,12613,12721,1,3.0
3,1,12646,12697,1,2.0
4,1,13221,14409,1,67.0


## Map exons to bs

In [28]:
#for a large data set, you will likely get a significant speed increase by creating an index for any column(s) used in the join condition. 
#https://stackoverflow.com/questions/30627968/merge-pandas-dataframes-where-one-value-is-between-two-others
#instead use SQL
import sqlite3
conn = sqlite3.connect(':memory:') #Make the database in memory

#write the tables
important_counts.to_sql('bs', conn, index=False, dtype={"bs_id": 'INTEGER'})
new_exon_counts.to_sql('counts', conn, index=True, index_label='exon_id', dtype={"raw_counts": 'INTEGER', "exon_id": 'INTEGER'})
qry = '''
    select
        counts.chromosome_name, counts.chrom_exon_start, counts.chrom_exon_end, counts.strand, counts.raw_counts, counts.exon_id, bs.bs_id, bs.miRNA, bs.mRNA, bs.chrom_bs_start, bs.chrom_bs_end
    from
        bs inner join counts on
        bs.chromosome_name = counts.chromosome_name and bs.strand = counts.strand and bs.chrom_bs_start >= counts.chrom_exon_start and bs.chrom_bs_end <= counts.chrom_exon_end
    '''
bs_exons = pd.read_sql_query(qry, conn) #only return bs with minimum 1 exon_id cause inner join (else left join)
bs_exons.head()
#TODO filter out all binding sites that have more than 1 exon_id: none for now

,chromosome_name,chrom_exon_start,chrom_exon_end,strand,raw_counts,exon_id,bs_id,miRNA,mRNA,chrom_bs_start,chrom_bs_end
0,19,58352928,58353197,-1,0,216603,2,hsa-let-7b-5p,ENST00000600966,58352987,58353006
1,19,58351391,58351687,-1,0,216600,3,hsa-let-7b-5p,ENST00000600966,58351458,58351475
2,19,58352283,58352555,-1,0,216602,4,hsa-let-7b-5p,ENST00000600966,58352400,58352420
3,19,58351391,58351687,-1,0,216600,5,hsa-let-7b-3p,ENST00000600966,58351648,58351684
4,19,58351391,58351687,-1,0,216600,8,hsa-let-7d-5p,ENST00000600966,58351399,58351420


In [158]:
#visualize amount exons by bindingsite | amount bindingsites by exon
import sqlite3
conn = sqlite3.connect(':memory:') #Make the database in memory

#write the tables
important_counts.to_sql('bs', conn, index=False, dtype={"bs_id": 'INTEGER'})
new_exon_counts.to_sql('counts', conn, index=True, index_label='exon_id', dtype={"exon_id": 'INTEGER'})
qry = '''
    select
        counts.chromosome_name, counts.chrom_exon_start, counts.chrom_exon_end, counts.strand, counts.raw_counts, counts.exon_id, bs.bs_id, bs.miRNA, bs.mRNA, bs.chrom_bs_start, bs.chrom_bs_end
    from
        bs left join counts on
        bs.chromosome_name = counts.chromosome_name and bs.strand = counts.strand and bs.chrom_bs_start >= counts.chrom_exon_start and bs.chrom_bs_end <= counts.chrom_exon_end
    '''
bs_exons2 = pd.read_sql_query(qry, conn)
bs_exons2.head()
#bs_exons2.groupby('bs_id').count().raw_counts.hist()
#bs_exons2.groupby('exon_id').count()

## Map miRNA to bs

In [29]:
#map miRNA ID to mature ID
mirna_counts = all_mirna_dfs[case_id]

trans = pd.DataFrame.from_dict(read_in_miRNA(path2+'miRNA2mature.fasta'), orient='index')#,columns=["mirna", "mature"])
trans = trans.rename(columns={0: "miRNA_ID"})
trans[['miRNA_ID']] = trans[['miRNA_ID']].astype('str').astype('category')

#map miRNA to miRNA family
mirna_families = read_in_mirna_families(path_tarp+'mapping_miRNA_families.fasta')
trans = trans.join(mirna_families,how='inner')

bs_mirnas = bs_exons[['exon_id','raw_counts','miRNA']].join(trans,on='miRNA').merge(mirna_counts[['miRNA_ID','read_count']],on='miRNA_ID')
#bs_mirnas.drop(['miRNA','miRNA_ID'], axis=1, inplace=True)
bs_mirnas = bs_mirnas.rename(columns={'read_count': "mirna_read_count",'raw_counts':'exon_raw_counts','miRNA family name':'mirna_family'})
bs_mirnas[['mirna_read_count']] = bs_mirnas[['mirna_read_count']].astype('int')#TODO do this early before saving
bs_mirnas.head()

#2693 with hsa in miRNA2mature / 49156 overall in miRNA2mature (also other species eg cel)
#2656 if inner join cause only 2656 families
#TODO check if miRNA counts from TCGA are more different miRNAs?
#TODO is inner right? are some in important counts None?

KeyError: '24ea3cd5-c2e6-44f7-9658-78af7befeef7'

In [227]:
bs_mirnas[bs_mirnas.exon_id ==216595]

,exon_id,exon_raw_counts,miRNA,miRNA_ID,mirna_family,mirna_read_count
7,216595,0,hsa-let-7b-5p,hsa-let-7b,let-7,55647
14,216595,0,hsa-let-7d-5p,hsa-let-7d,let-7,3767
25,216595,0,hsa-let-7e-3p,hsa-let-7e,let-7,11424
33,216595,0,hsa-let-7e-3p,hsa-let-7e,let-7,11424
46,216595,0,hsa-miR-15a-3p,hsa-mir-15a,mir-15,2258
...,...,...,...,...,...,...
12562,216595,0,hsa-miR-5692a,hsa-mir-5692a-2,mir-5692,0
12563,216595,0,hsa-miR-548az-5p,hsa-mir-548az,mir-548,0
12566,216595,0,hsa-miR-8062,hsa-mir-8062,hsa-miR-8062,0
12571,216595,0,hsa-miR-8064,hsa-mir-8064,hsa-miR-8064,0


In [226]:
bs_mirnas[bs_mirnas.mirna_family =='hsa-miR-1229-5p']

# 1 right? count is added for exonid 216595, but do i already add it before ?
# 2 also : exon_id + mirna double (3368 & 3370) wrong!!

,exon_id,exon_raw_counts,miRNA,miRNA_ID,mirna_family,mirna_read_count
3365,216603,0,hsa-miR-1229-5p,hsa-mir-1229,hsa-miR-1229-5p,4
3368,216595,0,hsa-miR-1229-5p,hsa-mir-1229,hsa-miR-1229-5p,4
3370,216595,0,hsa-miR-1229-5p,hsa-mir-1229,hsa-miR-1229-5p,4
3371,216596,0,hsa-miR-1229-5p,hsa-mir-1229,hsa-miR-1229-5p,4


In [ ]:
#output: for each case_id for each bs_id: interesting miRNA expression + exon expression
#unstack family counts from 2 columns to several columns (mirna_family=name,mirna_read_count=value)
pivoted = bs_mirnas.copy()
pivoted = pivoted.pivot_table(values='mirna_read_count', index=['exon_id', 'exon_raw_counts'], columns='mirna_family', aggfunc='sum', fill_value=0)
pivoted#.head()

In [ ]:
#visualize amount exons by bindingsite
#bs_exons.groupby('bs_id').count().raw_counts.hist()

In [274]:
#TODO check - is nr mirna downloaded also here? if not: mistake
print(X.shape,y[0].shape)

(8, 1680) (8,)


## Elastic Net Regression

In [ ]:
import sys
sys.getsizeof(pivoted)#TODO

In [276]:
#regression btw miRNA expression, exon expression
import sklearn
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
#faster if np.array(x, order='F')
#If you are interested in controlling the L1 and L2 penalty separately, keep in mind that this is equivalent to:
#a * L1 + b * L2 where: alpha = a + b and l1_ratio = a / (a + b)
X = np.array(pivoted)
y = np.array(pivoted.index.get_level_values('exon_raw_counts'))

regr = ElasticNet(random_state=0)
regr.fit(X, y)
print(regr.coef_)
print(regr.intercept_)

[0. 0. 0. ... 0. 0. 0.]
0.0


C:\Users\Lena\Miniconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.enet_coordinate_descent(


## Get exonid for exon for database later

In [ ]:
#TODO this is not necessary anymore if the new exon counts work!!!!
#instead try directly with ensembl start & end match - if it doesnt work 
#-> try download from https://www.gencodegenes.org/human/ (GRCh38.p13) if same reference genome

In [23]:
#read in exon ids + starts and ends from GENCODE GTF file (ca 12min)
#import the file into a local sqlite3 file-based database
import gffutils
fn = gffutils.example_filename(path2+'gencode.v19.annotation.gtf_withproteinids')
db = gffutils.create_db(fn, ':memory:', keep_order=True, disable_infer_genes=True, disable_infer_transcripts=True)

In [20]:
new_exon_counts['exonname'] = np.nan 

In [27]:
for exon in db.features_of_type('exon'):
    print(exon)#TODO
    is_same = (new_exon_counts.chromosome_name == exon.chromosome) and (new_exon_counts.chrom_exon_start == exon.start) and (new_exon_counts.chrom_exon_end == exon.stop) and (new_exon_counts.strand == exon.strand)
    new_exon_counts[is_same].exonname = exon.exon_id
    print(len(new_exon_counts[is_same]))
    #exons = list(db.children(gene, featuretype='exon'))
    #first_3_exons = exons[:3]
    #last_3_exons = exons[-3:]

chr1	HAVANA	exon	11869	12227	.	+	.	gene_id "ENSG00000223972.4"; transcript_id "ENST00000456328.2"; gene_type "pseudogene"; gene_status "KNOWN"; gene_name "DDX11L1"; transcript_type "processed_transcript"; transcript_status "KNOWN"; transcript_name "DDX11L1-002"; level "2"; havana_gene "OTTHUMG00000000961.2"; tag "basic"; havana_transcript "OTTHUMT00000362751.1"; exon_number "1"; exon_id "ENSE00002234944.1";


AttributeError: 'Feature' object has no attribute 'chromosome'

In [ ]:
#exon line
#chr1	HAVANA	exon	11869	12227	.	+	.	gene_id "ENSG00000223972.4"; 
#transcript_id "ENST00000456328.2"; gene_type "pseudogene"; gene_status "KNOWN"; 
#gene_name "DDX11L1"; transcript_type "processed_transcript"; transcript_status "KNOWN"; 
#transcript_name "DDX11L1-002"; exon_number 1; exon_id "ENSE00002234944.1"; level 2; tag "basic"; 
#havana_gene "OTTHUMG00000000961.2"; havana_transcript "OTTHUMT00000362751.1";